 ### Point-in-polygon (PIP) queries

Finding out if a certain point is located inside or outside of an area, or finding out if a line intersects with another line or polygon are fundamental geospatial operations that are often used e.g. to select data based on location. Such spatial queries are one of the typical first steps of the workflow when doing spatial analysis. Performing a spatial join (will be introduced later) between two spatial datasets is one of the most typical applications where Point in Polygon (PIP) query is used.

Computationally, detecting if a point is inside a polygon is most commonly done using a specific formula called Ray Casting algorithm. Luckily, we do not need to create such a function ourselves for conducting the Point in Polygon (PIP) query. Instead, we can take advantage of Shapely’s binary predicates that can evaluate the topolocical relationships between geographical objects, such as the PIP as we’re interested here.

###  Point-in-polygon queries on shapely geometries

There are basically two ways of conducting PIP in Shapely:

using a function called within() that checks if a point is within a polygon

using a function called contains() that checks if a polygon contains a point

Let’s first create a couple of point geometries:

In [ ]:
import shapely.geometry
point1 = shapely.geometry.Point(24.952242, 60.1696017)
point2 = shapely.geometry.Point(24.976567, 60.1612500)

… and a polygon:

In [ ]:
polygon = shapely.geometry.Polygon(
    [
        (24.950899, 60.169158),
        (24.953492, 60.169158),
        (24.953510, 60.170104),
        (24.950958, 60.169990)
    ]
)

In [ ]:
print(point1)
print(point2)
print(polygon)

Let’s check if the points are within() the polygon:

In [ ]:
point1.within(polygon)

In [ ]:
point2.within(polygon)

It seems that the first point is inside the polygon, but the second one is not.

We can turn the logic of the look-up around: Rather than check of the point is within the polygon, we can also ask whether the polygon contains() the point:

In [ ]:
polygon.contains(point1)

In [ ]:
polygon.contains(point2)

The two ways of checking the spatial relationship are complementary and yield equivalent results; contains() is inverse to within(), and vice versa.

Then, which one should you use? Well, it depends:

if you have many points and just one polygon and you try to find out which one of them is inside the polygon: You might need to iterate over the points and check one at a time if it is within() the polygon.

if you have many polygons and just one point and you want to find out which polygon contains the point: You might need to iterate over the polygons until you find a polygon that contains() the point specified

 ###  Point-in-polygon queries on geopandas.GeoDataFrames

In the following practical example we find which of the addresses we obtained in the geocoding section are located within a certain city district of Helsinki.

The data set we are using is from Helsinki Region Infoshare, and licensed under a Creative-Commons-Attribution-4.0 license.

In [ ]:
import pathlib
NOTEBOOK_PATH = pathlib.Path().resolve()
DATA_DIRECTORY = NOTEBOOK_PATH / "data"

In [ ]:
import geopandas

regions = geopandas.read_file(
    DATA_DIRECTORY / "se_100km.shp"
)
regions.head()

In [ ]:
regions.plot()

In [ ]:
regions = regions[regions.CELLCODE == "100kmE47N40"]
regions

In [ ]:
addresses = geopandas.read_file(DATA_DIRECTORY / "addresses.gpkg")

In [ ]:
axes = regions.plot(facecolor="grey")
regions.plot(ax=axes, facecolor="red")
addresses.plot(ax=axes, color="blue", markersize=5)

Some points are within the ‘Stockholm’ region, but others are not. To find out which are the ones inside the district, we can use a point-in-polygon query, this time on the entire geopandas.GeoDataFrame. Its method within() returns Boolean (True/False) values that indicate whether or not a row’s geometry is contained in the supplied other geometry:

In [ ]:
addresses.within(regions.geometry.iloc[0])

In [ ]:
addresses_in_stockholm = addresses[
    addresses.within(regions.geometry.iloc[0])
]
addresses_in_stockholm

Finally, let’s plot this list of addresses one more time to visually verify that all of them, indeed, are located within Stockholm:

In [ ]:
axes = regions.plot(facecolor="grey")
regions.plot(ax=axes, facecolor="red")

addresses_in_stockholm.plot(
    ax=axes,
    color="gold",
    markersize=5
)